**Clustering Toronto neighborhoods based on their characteristics**

In [38]:
# import libraries

import pandas as pd
import numpy as np
#!pip install geocoder
import geocoder
import requests
import folium as fl
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

**Continuing from the notebook for Q2, dfFinal shows the postal codes, boroughs, neighborhoods, and lat/lon**

In [164]:
dfFinal

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [178]:
# let's plot the neighborhoods in folium before clustering

import folium as fl

TorontoMap = fl.Map(location = [43.715,-79.3832],zoom_start=11)
postalcodes = fl.FeatureGroup().add_to(TorontoMap)

for code, lat, lon in zip(dfFinal["PostalCode"],dfFinal["Latitude"],dfFinal["Longitude"]):
    code = code
    lat = lat
    lon = lon
    postalcodes.add_child(fl.CircleMarker(location=[lat,lon],popup=code,tooltip=code,radius=10,fill=True,fill_opacity=.75))
TorontoMap

In [39]:
# get data on venues from Foursquare

CLIENT_ID = 'MY_CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'MY_CLIENT_SECRET' # your Foursquare Secret
VERSION = '20210301' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [89]:
# practice on the first Postal Code

M1Blat = dfFinal.loc[0,"Latitude"]
M1Blon = dfFinal.loc[0,"Longitude"]

url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={M1Blat},{M1Blon}&radius=1000&limit={LIMIT}"

results = requests.get(url).json()

In [73]:
results["response"]["groups"][0]["items"]

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4d669cba83865481c948fa53',
   'name': 'Images Salon & Spa',
   'location': {'address': '8130 Sheppard Ave E',
    'crossStreet': 'Morningside Ave',
    'lat': 43.80228301948931,
    'lng': -79.19856472801668,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.80228301948931,
      'lng': -79.19856472801668}],
    'distance': 595,
    'postalCode': 'M1B 3W3',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
     'Toronto ON M1B 3W3',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d1ed941735',
     'name': 'Spa',
     'pluralName': 'Spas',
     'shortName': 'Spa',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'coun

In [90]:
# extract category of each venue

def getcat(row):
    try:
        categories_list = row["categories"]
    except:
        categories_list = row["venue.categories"]
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]["name"]

In [91]:
venues = results["response"]["groups"][0]["items"] 
    # make sure results has groups and items as these seem to change
    # basically you're trying to remove extra layers of info so you can get at the useful information
    
nearby_venues = pd.json_normalize(venues) # flatten the JSON file

# filter the columns
filcols = ["venue.name","venue.categories","venue.location.lat","venue.location.lng"]
    # calling these columns comes from the heirarchy in results
    
nearby_venues = nearby_venues.loc[:,filcols]

# get category for each row
nearby_venues["venue.categories"] = nearby_venues.apply(getcat, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Harvey's,Restaurant,43.800020,-79.198307
2,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
3,Wendy's,Fast Food Restaurant,43.802008,-79.198080
4,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


**Clearly M1B is a hoppin' neighborhood**

In [107]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius=500&limit={LIMIT}"
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [108]:
# now get the venues in Toronto

TorontoVenues = getNearbyVenues(names = dfFinal["PostalCode"], 
                                latitudes = dfFinal["Latitude"], 
                                longitudes = dfFinal["Longitude"])

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [109]:
TorontoVenues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [117]:
TorontoVenues.shape

(2121, 7)

In [111]:
# groupby neighborhood to see how many venues categories exist in each one

TorontoVenuesGrouped = TorontoVenues.groupby("Neighborhood").count()
TorontoVenuesGrouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M1B,1,1,1,1,1,1
M1C,1,1,1,1,1,1
M1E,8,8,8,8,8,8
M1G,4,4,4,4,4,4
M1H,8,8,8,8,8,8
...,...,...,...,...,...,...
M9N,3,3,3,3,3,3
M9P,8,8,8,8,8,8
M9R,4,4,4,4,4,4


In [113]:
# find out how many unique categories there are

categorycount = len(TorontoVenues["Venue Category"].unique())

print(f"There are {categorycount} venue categories in Toronto")

There are 274 venue categories in Toronto


**Now I need to analyze the venues in each neighborhood. I'll start by one-hot encoding the data from TorontoVenues, grouping the resulant df by neighborhood and getting the mean venues for each venue category in each neighborhood.**

In [146]:
# one-hot encode and clean the df
torontodummies = pd.get_dummies(TorontoVenues[["Venue Category"]],prefix="",prefix_sep="") # one-hot encode
torontodummies.drop("Neighborhood",axis=1,inplace=True) # neighborhood is in the middle which is inconvenient
torontodummies.insert(0,"Neighborhood",TorontoVenues["Neighborhood"]) # add it back to the front
torontodummies.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
# groupby neighborhood with mean

torontodummiesgrouped = torontodummies.groupby("Neighborhood").mean().reset_index()

torontodummiesgrouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000

**Now that the neighborhoods have been given values for each type of venue in their area, they can be categorized and, later, clustered. I'll look at the 10 most common venue types in each neighborhood below**

In [153]:
# find the n most common venues in each neighborhood

def findmostcommonvenues(row,nvenues):
    categories = row.iloc[1:] # gets all categories for that row
    sortedcats = categories.sort_values(ascending=False) # sorts in descending order
    return sortedcats.index.values[0:nvenues] # return the top n venue categories for that row

    
nvenues = 10

# create a df

columns = ["First","Second","Third","Fourth","Fifth",
          "Sixth","Seventh","Eighth","Ninth","Tenth"]

mostcommonvenues = pd.DataFrame(columns=columns)
mostcommonvenues.insert(0,"Neighborhood",torontodummiesgrouped["Neighborhood"])

for i in np.arange(torontodummiesgrouped.shape[0]): # len of the df
    mostcommonvenues.iloc[i,1:] = findmostcommonvenues(torontodummiesgrouped.iloc[i,:], nvenues)

mostcommonvenues.head()

,Neighborhood,First,Second,Third,Fourth,Fifth,Sixth,Seventh,Eighth,Ninth,Tenth
0,M1B,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Hakka Restaurant
1,M1C,Bar,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
2,M1E,Medical Center,Intersection,Restaurant,Breakfast Spot,Electronics Store,Rental Car Location,Bank,Mexican Restaurant,Distribution Center,Dog Run
3,M1G,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Caribbean Restaurant,Bakery,Gas Station,Thai Restaurant,Athletics & Sports,Fried Chicken Joint,Bank,Hakka Restaurant,Eastern European Restaurant,Dumpling Restaurant


In [191]:
# now we have info about the makeup of each neighborhood. Time to cluster

k = 7

torontoclusteringdf = torontodummiesgrouped.drop("Neighborhood",axis=1)

kmeans = KMeans(n_clusters = k, random_state=0) # create model object
kmeans.fit(torontoclusteringdf) # fit data to the model
kmeans.labels_[0:10] # display the first 10 labels

array([1, 0, 1, 1, 1, 4, 1, 1, 1, 1], dtype=int32)

**I went with 7 clusters after some trial and error. The short story is a lot of the "neighborhoods" are similar because they're too close together**

In [192]:
# insert the cluster label into the most common venues df

mappingdf = pd.DataFrame() # create new df for mapping clusers

mappingdf.insert(0,"Cluster",kmeans.labels_) # cluster label col
mappingdf.insert(0,"PostalCode",torontodummiesgrouped["Neighborhood"]) # postalcode col

# get lat/lon of each postal code

mappingdf = mappingdf.merge(dfFinal[["Latitude","Longitude","PostalCode","Neighborhood"]],on="PostalCode")

mappingdf.head()

,PostalCode,Cluster,Latitude,Longitude,Neighborhood
0,M1B,1,43.806686,-79.194353,"Malvern, Rouge"
1,M1C,0,43.784535,-79.160497,"Rouge Hill, Port Union, Highland Creek"
2,M1E,1,43.763573,-79.188711,"Guildwood, Morningside, West Hill"
3,M1G,1,43.770992,-79.216917,Woburn
4,M1H,1,43.773136,-79.239476,Cedarbrae


**Now I have the postalcodes labeled (based on their cluster) and their lat/lon back in the dataframe. Now I need to visualize**

In [193]:
# map the neighborhoods with colors based on cluster

colors = ["firebrick","coral","goldenrod","yellow","turquoise","darkgreen","dodgerblue","midnightblue","mediumpurple","hotpink","chartreuse","black","white","slategray","orange"]

clusteredmap = fl.Map(location = [43.715,-79.3832],zoom_start=11)

markers = fl.FeatureGroup().add_to(clusteredmap)

for lat, lon, name, cluster in zip(mappingdf["Latitude"],
                                  mappingdf["Longitude"],
                                  mappingdf["PostalCode"],
                                  mappingdf["Cluster"]):
    markers.add_child(fl.CircleMarker([lat,lon],radius=5,popup=f"{name} - {cluster}",
                    color=colors[cluster],fill=True,
                    fill_color=colors[cluster],
                   fill_opacity=.8))
    
clusteredmap

**And there you have it. If I had to do it over, I would use actual neighborhoods instead of postal codes. The postal codes are too numerous and close together so they cover a lot of the same venues and we end up with a lot of "neighborhoods" in the same cluster. This is fine, but it could be done better. For my final project for the course, I'll follow this same formula but use my local city's neighborhoods instead. This should give a better look at the areas and the venues therein.**